In [1]:
import numpy as np
import pandas as pd
from fredmd import FredMD
import sklearn.pipeline as skpipe
import sklearn.decomposition as skd
import sklearn.preprocessing as skp
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.utils.extmath import randomized_svd

In [2]:
from datetime import datetime

# Data Loaders

In [730]:
class SPCAData:
    
    def __init__(self, Nfactor=None, vintage=None, maxfactor=8, standard_method=2, ic_method=2,
                 target=None, train_test_split=[('1960-01-01', '1984-12-01'),('1985-01-01', '2019-12-01')], 
                 nlags=1, drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]) -> None:
        """
        Create fredmd object
        Auguments:
        1) Nfactor = None: Number of factors to estimate. If None then estimate number of true factors via information critea
        2) vintage = None: Vinatege of data to use in "year-month" format (e.g. "2020-10"). If None use current vintage
        3) maxfactor = 8: Maximimum number of factors to test against information critea. If Nfactor is a number, then this is ignored
        4) standard_method = 2: method to standardize data before factors are estimate. 0 = Identity transform, 1 = Demean only, 2 = Demean and stardize to unit variance. Default = 2.
        5) ic_method = 2: information critea penalty term. Se
        e http://www.columbia.edu/~sn2294/pub/ecta02.pdf page 201, equation 9 for options.
        """
        self.drop_cols = drop_cols
        self.train_test_split = [[datetime.strptime(x, '%Y-%m-%d') for x in split] for split in train_test_split]
        # Make sure arguments are valid
        if standard_method not in [0, 1, 2]:
            raise ValueError(f"standard_method must be in [0, 1, 2], got {standard_method}")
        if ic_method not in [1, 2, 3]:
            raise ValueError(f"ic_method must be in [1, 2, 3], got {ic_method}")
        # Download data
        self.rawseries, self.transforms, self.target, self.train_mask, self.test_mask = self.download_data(vintage, 
                                                                       target if target is not None else "UNRATE",
                                                                         self.train_test_split)
        

        self.target_name = target
        self.standard_method = standard_method
        self.ic_method = ic_method
        self.maxfactor = maxfactor
        self.Nfactor = Nfactor

        self.nlags = nlags

    @staticmethod
    def download_data(vintage, tgt, train_test_split):
        if vintage is None:
            url = 'https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/current.csv'
        else:
            url = f'https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/{vintage}.csv'
        transforms = pd.read_csv(
            url, header=0, nrows=1, index_col=0).transpose()
        transforms.index.rename("series", inplace=True)
        transforms.columns = ['transform']
        transforms = transforms.to_dict()['transform']
        data = pd.read_csv(url, names=transforms.keys(), skiprows=2, index_col=0,
                           skipfooter=1, engine='python', parse_dates=True, infer_datetime_format=True)
        
        train_mask = (data.index > train_test_split[0][0]) & (data.index <= train_test_split[0][1])
        test_mask = (data.index > train_test_split[1][0]) & (data.index <= train_test_split[1][1])
        
        target = None
        if "FB-yeild" in tgt:
            bond_data = pd.read_csv("data/bond_data.csv", engine='python', parse_dates=True, infer_datetime_format=True,
                       skiprows=range(1,45), index_col=4)
            if tgt == 'FB-yeild-1':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5001]
            elif tgt == 'FB-yeild-2':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5002]
            elif tgt == 'FB-yeild-3':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5003]
            elif tgt == 'FB-yeild-4':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5004]
            elif tgt == 'FB-yeild-5':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5005]
            bond_data.index = bond_data.index.to_period('M') 
            intersection = bond_data.index.intersection(data.index.to_period('M'))
            target = bond_data[bond_data.index.isin(intersection)]["TMYTM"]
            
        return data, transforms, target, train_mask, test_mask

    @staticmethod
    def factor_standardizer_method(code):
        """
        Outputs the sklearn standard scaler object with the desired features
        codes:
        0) Identity transform
        1) Demean only
        2) Demean and standardized
        """
        if code == 0:
            return skp.StandardScaler(with_mean=False, with_std=False)
        elif code == 1:
            return skp.StandardScaler(with_mean=True, with_std=False)
        elif code == 2:
            return skp.StandardScaler(with_mean=True, with_std=True)
        else:
            raise ValueError("standard_method must be in [0, 1, 2]")

    @staticmethod
    def data_transforms(series, transform):
        """
        Transforms a single series according to its transformation code
        Inputs:
        1) series: pandas series to be transformed
        2) transfom: transform code for the series
        Returns:
        transformed series
        """
        if transform == 1:
            # level
            return series
        elif transform == 2:
            # 1st difference
            return series.diff()
        elif transform == 3:
            # second difference
            return series.diff().diff()
        elif transform == 4:
            # Natural log
            return np.log(series)
        elif transform == 5:
            # log 1st difference
            return np.log(series).diff()
        elif transform == 6:
            # log second difference
            return np.log(series).diff().diff()
        elif transform == 7:
            # First difference of percent change
            return series.pct_change().diff()
        else:
            raise ValueError("Transform must be in [1, 2, ..., 7]")

    def apply_transforms(self):
        """
        Apply the transformation to each series to make them stationary and drop the first 2 rows that are mostly NaNs
        Save results to self.series
        """
        self.series = pd.DataFrame({key: self.data_transforms(
            self.rawseries[key], value) for (key, value) in self.transforms.items()})

    def remove_outliers(self):
        """
        Removes outliers from each series in self.series
        Outlier definition: a data point x of a series X is considered an outlier if abs(x-median)>10*interquartile_range.
        """
        Z = abs((self.series - self.series.median()) /
                (self.series.quantile(0.75) - self.series.quantile(0.25))) > 10
        for col, _ in self.series.iteritems():
            self.series[col][Z[col]] = np.nan
        

    def get_data(self):
        """
        """
        # Define our estimation pipelines
        self.apply_transforms()
        self.remove_outliers()
        
        self.series = self.series.loc[self.series.index >= '1960-01-01']
        self.series = self.series.loc[self.series.index < '2020-01-01']
        
        pipe = skpipe.Pipeline([('Standardize', self.factor_standardizer_method(self.standard_method))])

        actual_data = self.series.to_numpy(copy=True)
        intial_nas = self.series.isna().to_numpy(copy=True)
        working_data = self.series.fillna(value=self.series.mean(), axis='index').to_numpy(copy=True)

        last_timestep = np.sum(self.train_mask) + np.sum(self.test_mask)     

        if self.target is None:
            idx = self.series.columns.get_loc(self.target_name)
            target_data = np.copy(working_data[:,idx])

        else:
            target_data = self.target
            target_data = target_data.loc[target_data.index >= '1960-01-01']
            target_data = target_data.loc[target_data.index < '2020-01-01'].to_numpy(copy=True)
        
        bad_cols = np.isin(self.series.columns.to_numpy(), self.drop_cols)
        
        return working_data[:-1,~bad_cols], target_data[1:], np.sum(self.train_mask), working_data[:last_timestep]
        


In [623]:
class ChenZData:
    
    def __init__(self, start_date='1976-03', end_date='2019-09', tgt_factor='mkt'):
        chen_z_port_data = pd.read_csv("data/allportretbase.csv", engine='python', 
                                       parse_dates=True, infer_datetime_format=True,
                               index_col=0)
        chen_z_port_data = chen_z_port_data.loc[chen_z_port_data.date >= start_date]
        chen_z_port_data = chen_z_port_data.loc[chen_z_port_data.date < end_date]

        chen_z_port_data['port_id'] = chen_z_port_data.signalname.add(chen_z_port_data.port.astype(str))
        chen_z_port_data.port_id = chen_z_port_data.port_id
        portfolios = sorted(chen_z_port_data.port_id.unique())
        dates = chen_z_port_data.date.unique()
        portfolios = [p for p in portfolios if 
                     np.sum(chen_z_port_data['port_id'] == p) == len(dates)]
        return_panel = np.zeros((len(dates), len(portfolios)), dtype=float)
        for i, pname in enumerate(portfolios):
            return_panel[:, i] = chen_z_port_data.loc[chen_z_port_data['port_id'] == pname].ret.to_numpy()
        tgt = None
        if tgt_factor in {'mkt', 'SMB', 'HML', 'CMA', 'RMW'}:
            int_start_date = int(start_date.replace('-', ''))
            int_end_date = int(end_date.replace('-', ''))
            ff_factors = pd.read_csv('data/F-F_Research_Data_5_Factors_2x3.CSV', skiprows=0,
                                       engine='python', parse_dates=True, infer_datetime_format=True)
            ff_factors = ff_factors.loc[ff_factors.date >= int_start_date] #NOTE not offsetting by one here across data sources
            ff_factors = ff_factors.loc[ff_factors.date < int_end_date]
            if tgt_factor == 'mkt':
                tgt = ff_factors['Mkt-RF'].to_numpy()
            if tgt_factor == 'SMB':
                tgt = ff_factors['SMB'].to_numpy()
            if tgt_factor == 'HML':
                tgt = ff_factors['HML'].to_numpy()
            if tgt_factor == 'CMA':
                tgt = ff_factors['CMA'].to_numpy()
            if tgt_factor == 'RMW':
                tgt = ff_factors['RMW'].to_numpy()
        self.data = return_panel, tgt, int(tgt.shape[0]/2)

# Models


##  SPCA

In [763]:
def scale(X, ref=None):
    if ref is None:
        ref = X
    norm = np.std(ref, axis=0, keepdims=True)
    return (X - np.mean(ref, axis=0, keepdims=True)) / norm

class SPCA:
    
    def __init__(self, data_panel, target_panel, n_train,
                N_factor=6) -> None:
        """
        Auguments:
        1) data_panel: numpy 2d array of data, normalizations and transforms should have already been applied
        2) target_panel: numpy 2d array of target data, normalizations and transforms should have already been applied
        3) n_train: index of start of oos
        4) N_factors: number of factors
        """
        self.n_factors = N_factor
        self.test_start = n_train
        self.data_series = np.copy(data_panel)
        self.tgt_series = np.copy(target_panel)
        
    def fit(self, nlags, true_oos=False, pca=False, raw_data=None,
           stack_lags=False, plot_resids=False, fit_factors_epanding_window=True):
        T, N = self.data_series.shape
        preds = []
        ar_preds = []
        gts = []
        mean_preds = []
        
        
        lags = np.zeros((T, nlags))
        if stack_lags:
            for t in range(T):
                lags[t,max(0, nlags-t):] = self.tgt_series[max(0, t-nlags):t]
                
        if not fit_factors_epanding_window:
            scaled_data = scale(np.copy(self.data_series[:self.test_start,:]))
            scaled_data_full = scale(np.copy(self.data_series), 
                                     ref=self.data_series[:self.test_start,:])
            gamma_is = SPCA.get_gamma_is(scaled_data, self.tgt_series[:self.test_start])
            if pca:
                gamma_is[:] = 1
            gamma_is = np.diag(gamma_is)
            _, loadings = SPCA.fit_factors(scaled_data@gamma_is, self.n_factors)
            
            factors = (1/N ) * scaled_data_full @ loadings
            
            
        for t in range(self.test_start, self.tgt_series.shape[0]):
            if fit_factors_epanding_window:
                scaled_data = scale(np.copy(self.data_series[:t,:]))
                scaled_data_ext = scale(np.copy(self.data_series[:t+1,:]))
                assert np.sum(np.isnan(scaled_data)) == 0 and np.sum(np.isnan(scaled_data_ext)) == 0 
                gamma_is = SPCA.get_gamma_is(scaled_data, self.tgt_series[:t])

                if pca:
                    gamma_is[:] = 1
                gamma_is = np.diag(gamma_is)
                if true_oos:
                    _, loadings = SPCA.fit_factors(scaled_data@gamma_is, self.n_factors)
                    factors = (1/N ) * scaled_data_ext @ loadings
                    fit_factors = factors[:t]
                    test_factors = factors[t:t+1]
                else:
                    factors, _ = SPCA.fit_factors(scaled_data_ext@gamma_is, self.n_factors, raw_data=raw_data)
                    fit_factors = factors[:t]
                    test_factors = factors[t:]
            else:
                fit_factors = factors[:t]
                test_factors = factors[t:t+1]
                
            
            if stack_lags:
                A_fit = np.concatenate([fit_factors, np.ones((t, 1)), lags[:t]], axis=1)
                A_test = np.concatenate([test_factors, np.ones((1, 1)), lags[t:t+1]], axis=1)
            else:
                A_fit = np.concatenate([fit_factors, np.ones((t, 1))], axis=1)
                A_test = np.concatenate([test_factors, np.ones((1, 1))], axis=1)
            loadings = np.linalg.lstsq(A_fit, self.tgt_series[:t], rcond=None)[0]

            
            sim_ar_model = AutoReg(self.tgt_series[:t], lags=nlags, old_names=False,
                                  trend='n')
            sim_ar_model_fit = sim_ar_model.fit()
            
            gts.append(self.tgt_series[t])
            mean_preds.append(np.mean(self.tgt_series[:t]))
            ar_forcast = sim_ar_model_fit.forecast()
            ar_preds.append(ar_forcast)
            preds.append(A_test@loadings)
            
        preds = np.array(preds).squeeze()
        ar_preds = np.array(ar_preds).squeeze()
        gts = np.array(gts)
        mean_preds = np.array(mean_preds)

        if plot_resids:
            plt.plot(preds - gts, label='factor resids')
            plt.plot(ar_preds - gts, label='ar resids')
            plt.legend()
            plt.show()
        
        print("r2 vs ar model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - ar_preds)))
        print("ar r2 vs mean model", 1 - np.sum(np.square(gts - ar_preds)) / np.sum(np.square(gts - mean_preds)))
        print("r2 vs mean model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - mean_preds)))
        print("EV", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts)))
        

        
    def fit_ts_reg(self, plot_resids=False):
        self.test_start
        gamma_is = SPCA.get_gamma_is(self.data_series[:self.test_start,:], self.tgt_series[:self.test_start])
        factors = SPCA.fit_factors(self.data_series*gamma_is, self.n_factors)[self.test_start:]
        loadings = np.linalg.lstsq(factors, self.tgt_series[self.test_start:], rcond=None)[0]
        preds = factors.dot(loadings)
        gts = self.tgt_series[self.test_start:]
        if plot_resids:
            plt.plot(preds - gts, label='factor resids')
            plt.legend()
            plt.show()
        
        print("r2 vs mean model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - np.mean(gts))))
        print("EV", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts)))
    

    @staticmethod
    def fit_factors(scaled_data, n_factors):
        T,N = scaled_data.shape

        fit_pipe = skpipe.Pipeline([('loadings', skd.TruncatedSVD(n_factors, algorithm='arpack'))])
        objective = scaled_data.T.dot(scaled_data)
        loadings = fit_pipe.fit_transform(objective)
        factors = (1/N ) * scaled_data.dot(loadings)
        
        return factors, loadings
        
    
    @staticmethod
    def get_gamma_is(data, target):
        loadings = []
        T = target.shape[0]
        
        for i in range(data.shape[1]):
            A = np.stack([data[:,i], np.ones(T)], axis=1)
            loading = np.linalg.lstsq(A, target, rcond=None)[0][0]
            loadings.append(loading)
        return np.array(loadings)

    

In [777]:


class GX_SPCA:
    
    def __init__(self, data_panel, target_panel, test_start, N_factors=6):
        """
        Auguments:
        1) data_panel: numpy 2d array of data, normalizations and transforms should have already been applied
        2) target_panel: numpy 2d array of target data, normalizations and transforms should have already 
            been applied
        3) n_train: index of start of oos
        4) N_factors: number of factors
        """
        T,N = data_panel.shape
        self.train_data = scale(data_panel[:test_start])
        self.train_target = target_panel[:test_start]
        self.test_data = scale(data_panel[test_start:], ref=data_panel[:test_start])
        self.test_target = target_panel[test_start:]
        self.n_factors = N_factors
        
    def fit(self, nlags, true_oos=False, pca=False, raw_data=None,
           stack_lags=False, plot_resids=False, quantile=None, print_res=False):
        
        if quantile is None:
            qtile_test_start = self.train_data.shape[0]//2
            quantiles = [i*(1/50) for i in range(50)]
            qtuile_r2s = [GX_SPCA(self.train_data, self.train_target, 
                                  qtile_test_start, self.n_factors).fit(nlags=nlags, true_oos=True,
                                                                       quantile=q)[0] for q in quantiles]
            quantile = quantiles[np.argmax(qtuile_r2s)]
            
        betas = GX_SPCA.get_portfolio_loadings(self.train_data, self.train_target, self.n_factors,
                                              quantile=quantile)
        
        train_factors = self.train_data @ betas
        test_factors = self.test_data @ betas
        
        preds = []
        mean_preds = []
        gts = []
        for t in range(self.test_data.shape[0]):
            if t > 0:
                fit_factors = np.concatenate([train_factors, test_factors[:t]], axis=0)
                fit_target = np.concatenate([self.train_target, self.test_target[:t]], axis=0)
            else:
                fit_factors = train_factors
                fit_target = self.train_target
            
            eval_factors = test_factors[t:t+1]
            
            A_fit = np.concatenate([fit_factors, np.ones((fit_factors.shape[0], 1))], axis=1)
            A_test = np.concatenate([eval_factors, np.ones((1, 1))], axis=1)
            
            loadings = np.linalg.lstsq(A_fit, fit_target, rcond=None)[0]
        
            gts.append(self.test_target[t])
            mean_preds.append(np.mean(np.concatenate([self.train_target,
                                                self.test_target[:t]], axis=0)))
        
            preds.append(A_test@loadings)
            
        preds = np.array(preds).squeeze()
        mean_preds = np.array(mean_preds)
        gts = np.array(gts)
        r2 =  1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - np.mean(mean_preds)))
        ev = 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts))
        if print_res:
            print("r2 vs mean model", r2)
            print("EV", ev)
        else:
            return r2, ev
        
    
    
    @staticmethod
    def get_portfolio_loadings(train_data, train_tgt, nfactor, quantile):
        T, N = train_data.shape
        loadings = np.zeros((N, nfactor))
        
        R_k = np.copy(train_data.T)
        G_k = np.copy(train_tgt.T)
        rbar_k = np.mean(R_k, axis=1)
        
        eta_hats = []
        gamma_hats = []
        v_hats = []
        beta_hats = []
        
        for k in range(nfactor):
            vhat, gamma_hat, eta_hat, beta_hat, R_k, rbar_k, G_k = GX_SPCA.get_next_factor_loading(R_k, G_k, rbar_k,
                                                                                                  quantile=quantile)
            eta_hats.append(eta_hat)
            gamma_hats.append(gamma_hat)
            v_hats.append(vhat)
            beta_hats.append(beta_hat)
        
        betas = np.concatenate(beta_hats, axis=1)
        return betas
        
    @staticmethod
    def get_next_factor_loading(train_data, train_tgt, rbar, quantile):
        #train_data: n times T
        #train_tgt: 1 times T
        N,T = train_data.shape 
        cors = (1/T)*(train_data @ train_tgt.T)
        thresh = np.quantile(cors, quantile) # FIXME
        selected_inds = np.argwhere(cors >= thresh).squeeze()
#         print(cors.shape, selected_inds.shape)
        selected_data = train_data[selected_inds,:] #
        
        # algo 1
        psi, singval, xi = randomized_svd(selected_data, n_components=1,
                                           n_iter=10, random_state=None)
        vhat = np.sqrt(T) * xi #factors
        gamma_hat = (singval/T)*psi.T @ rbar[selected_inds]
        eta_hat = (1/T) * train_tgt @ vhat.T
        #END algo 1
        
        beta_hat = (1/T) * train_data @ vhat.T
        
        train_data_perp = train_data - beta_hat @ vhat
        rbar_perp = rbar - beta_hat.squeeze() * gamma_hat
        target_perp = train_tgt - eta_hat * vhat.squeeze()
        
        return vhat, gamma_hat, eta_hat, beta_hat, train_data_perp, rbar_perp, target_perp

# Metrics

In [759]:
from statsmodels.tools.eval_measures import bic

In [760]:
def get_lags(target_series_train, max_lags):
    bics = []
    for i in range(max_lags):
        i += 1
        model = AutoReg(target_series_train, lags=i, old_names=False, trend='n')
        model_fit = model.fit()
        bics.append(model_fit.bic)
    return np.argmax(bics) + 1

In [779]:
for tgt in ["VXOCLSx", "UNRATE", "INDPRO", "CPIAUCSL"]:
    spca_data = SPCAData(target=tgt, 
                         drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]
#                          drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx"]
                        )
    data_panel, tgt_data, test_start, raw_data = spca_data.get_data()
    opt_lags = get_lags(tgt_data[:test_start], max_lags=5)
    print(tgt, opt_lags)
    spca = SPCA(data_panel, tgt_data, test_start, N_factor=4)
    print("SPCA")
    spca.fit(opt_lags, true_oos=True, stack_lags=False, fit_factors_epanding_window=False)
    print("PCA")
    spca.fit(opt_lags, true_oos=True, stack_lags=False, pca=True, fit_factors_epanding_window=False)
    print("giglio xui")
    spca = GX_SPCA(data_panel, tgt_data, test_start, N_factors=4)
    spca.fit(1, true_oos=True, stack_lags=False, print_res=True)
    print()

VXOCLSx 1
SPCA
r2 vs ar model -2.984284419953151
ar r2 vs mean model 0.77591764844881
r2 vs mean model 0.10719217792812852
EV 0.8684778894213432
PCA
r2 vs ar model -3.0016532633523756
ar r2 vs mean model 0.77591764844881
r2 vs mean model 0.10330012665550592
EV 0.867904540056339
giglio xui
r2 vs mean model 0.11110006384028492
EV 0.8704280855325958

UNRATE 1
SPCA
r2 vs ar model 0.14731217018472953
ar r2 vs mean model 0.003061115352460897
r2 vs mean model 0.14992234599143361
EV 0.14720830611166014
PCA
r2 vs ar model 0.0910458611534739
ar r2 vs mean model 0.003061115352460897
r2 vs mean model 0.09382827462257981
EV 0.09093514340215536
giglio xui
r2 vs mean model 0.08763491352427699
EV 0.08647088942449666

INDPRO 5
SPCA
r2 vs ar model -0.03915159331963913
ar r2 vs mean model 0.11179035309279373
r2 vs mean model 0.07701553021450258
EV 0.12154856388274626
PCA
r2 vs ar model -0.027505101440528312
ar r2 vs mean model 0.11179035309279373
r2 vs mean model 0.08736005665415525
EV 0.1313939777595124

In [772]:
for i in range(5):
    tgt = f'FB-yeild-{i+1}'
    spca_data = SPCAData(target=tgt, 
                         drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]
#                          drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx"]
                        )
    data_panel, tgt_data, test_start, raw_data = spca_data.get_data()
    opt_lags = get_lags(tgt_data[:test_start], max_lags=5)
    print(tgt, opt_lags)
    spca = SPCA(data_panel, tgt_data, test_start, N_factor=4)
    spca.fit(opt_lags, true_oos=True, stack_lags=False, fit_factors_epanding_window=False)
    spca.fit(opt_lags, true_oos=True, pca=True, fit_factors_epanding_window=False)
    print("giglio xui")
    for q in range(50):
        print("q", f'{q}/50')
        spca = GX_SPCA(data_panel, tgt_data, test_start, N_factors=4)
        spca.fit(1, true_oos=True, stack_lags=False, quantile=q/50)
    print()

FB-yeild-1 4
r2 vs ar model -121.13084488570136
ar r2 vs mean model 0.9942046744466398
r2 vs mean model 0.2922119937804297
EV 0.6139277027419132
r2 vs ar model -132.98450671689582
ar r2 vs mean model 0.9942046744466398
r2 vs mean model 0.22351616446921863
EV 0.5764566571729365
giglio xui
q 0/50
(123,) (123,)
(123,) (123,)
(123,) (123,)
(123,) (123,)
r2 vs mean model 0.35446818259255974
EV 0.5764564682782942
q 1/50
(123,) (120,)
(123,) (120,)
(123,) (120,)
(123,) (120,)
r2 vs mean model 0.34921189870202773
EV 0.573007738126357
q 2/50
(123,) (118,)
(123,) (118,)
(123,) (118,)
(123,) (118,)
r2 vs mean model 0.3269904802074768
EV 0.5584279175578726
q 3/50
(123,) (115,)
(123,) (115,)
(123,) (115,)
(123,) (115,)
r2 vs mean model 0.3276331004372617
EV 0.5588495507513719
q 4/50
(123,) (113,)
(123,) (113,)
(123,) (113,)
(123,) (113,)
r2 vs mean model 0.3047812458582897
EV 0.543856091197914
q 5/50
(123,) (110,)
(123,) (110,)
(123,) (110,)
(123,) (110,)
r2 vs mean model 0.2826454467887698
EV 0.52

r2 vs mean model 0.26338824726065624
EV 0.5714323352428035
q 15/50
(123,) (86,)
(123,) (86,)
(123,) (86,)
(123,) (86,)
r2 vs mean model 0.2557949794924741
EV 0.5670145004428782
q 16/50
(123,) (83,)
(123,) (83,)
(123,) (83,)
(123,) (83,)
r2 vs mean model 0.24730964267969824
EV 0.562077651459671
q 17/50
(123,) (81,)
(123,) (81,)
(123,) (81,)
(123,) (81,)
r2 vs mean model 0.23402593444430664
EV 0.5543490647291628
q 18/50
(123,) (79,)
(123,) (79,)
(123,) (79,)
(123,) (79,)
r2 vs mean model 0.2268981948655019
EV 0.5502020786202312
q 19/50
(123,) (76,)
(123,) (76,)
(123,) (76,)
(123,) (76,)
r2 vs mean model 0.21196163809435664
EV 0.5415118490235441
q 20/50
(123,) (74,)
(123,) (74,)
(123,) (74,)
(123,) (74,)
r2 vs mean model 0.21451668520248668
EV 0.5429983995278005
q 21/50
(123,) (71,)
(123,) (71,)
(123,) (71,)
(123,) (71,)
r2 vs mean model 0.21856462726332615
EV 0.5453535303950925
q 22/50
(123,) (69,)
(123,) (69,)
(123,) (69,)
(123,) (69,)
r2 vs mean model 0.21734364015334684
EV 0.544643148

r2 vs mean model 0.19301182949026618
EV 0.5842637872035856
q 32/50
(123,) (44,)
(123,) (44,)
(123,) (44,)
(123,) (44,)
r2 vs mean model 0.19463090604691913
EV 0.5850978871080059
q 33/50
(123,) (42,)
(123,) (42,)
(123,) (42,)
(123,) (42,)
r2 vs mean model 0.19655143529397479
EV 0.5860872864386445
q 34/50
(123,) (40,)
(123,) (40,)
(123,) (40,)
(123,) (40,)
r2 vs mean model 0.1989292210491861
EV 0.5873122506708816
q 35/50
(123,) (37,)
(123,) (37,)
(123,) (37,)
(123,) (37,)
r2 vs mean model 0.19826454417768435
EV 0.586969828990627
q 36/50
(123,) (35,)
(123,) (35,)
(123,) (35,)
(123,) (35,)
r2 vs mean model 0.2030765253597554
EV 0.5894488180338222
q 37/50
(123,) (32,)
(123,) (32,)
(123,) (32,)
(123,) (32,)
r2 vs mean model 0.20241952618445846
EV 0.5891103516734346
q 38/50
(123,) (30,)
(123,) (30,)
(123,) (30,)
(123,) (30,)
r2 vs mean model 0.20001124407877313
EV 0.5878696766319026
q 39/50
(123,) (27,)
(123,) (27,)
(123,) (27,)
(123,) (27,)
r2 vs mean model 0.19852104183568164
EV 0.587101968

FB-yeild-5 5
r2 vs ar model -107.43563375829123
ar r2 vs mean model 0.9903080633410175
r2 vs mean model -0.050951293961988275
EV 0.6481020375455577
r2 vs ar model -91.13208574420396
ar r2 vs mean model 0.9903080633410175
r2 vs mean model 0.10706166070722634
EV 0.7010107090596097
giglio xui
q 0/50
(123,) (123,)
(123,) (123,)
(123,) (123,)
(123,) (123,)
r2 vs mean model 0.2702450055537824
EV 0.7010106675506946
q 1/50
(123,) (120,)
(123,) (120,)
(123,) (120,)
(123,) (120,)
r2 vs mean model 0.2727828813990145
EV 0.7020504655795947
q 2/50
(123,) (118,)
(123,) (118,)
(123,) (118,)
(123,) (118,)
r2 vs mean model 0.2633916997674197
EV 0.6982027863608022
q 3/50
(123,) (115,)
(123,) (115,)
(123,) (115,)
(123,) (115,)
r2 vs mean model 0.26032885142718154
EV 0.6969479008340014
q 4/50
(123,) (113,)
(123,) (113,)
(123,) (113,)
(123,) (113,)
r2 vs mean model 0.2712179839423724
EV 0.7014093084111288
q 5/50
(123,) (110,)
(123,) (110,)
(123,) (110,)
(123,) (110,)
r2 vs mean model 0.2648903051738488
EV 0

In [773]:
for tgt in 'mkt', 'SMB', 'HML', 'CMA', 'RMW' :
    print(tgt)
    chen_z_data_panel, tgt_factor, weak_test_start = ChenZData(tgt_factor=tgt).data
    spca = SPCA(chen_z_data_panel, tgt_factor, weak_test_start, N_factor=5)
    spca.fit(1, true_oos=False, stack_lags=False, fit_factors_epanding_window=False)
    for q in range(50):
        print("q", f'{q}/50')
        spca = GX_SPCA(chen_z_data_panel, tgt_factor, weak_test_start, N_factors=5)
        spca.fit(1, true_oos=True, stack_lags=False, quantile=q/50)
    print()

mkt
r2 vs ar model 0.9729738598080268
ar r2 vs mean model -0.008956470002243533
r2 vs mean model 0.9727318009941209
EV 0.9730444366094002
q 0/50
(772,) (772,)
(772,) (772,)
(772,) (772,)
(772,) (772,)
(772,) (772,)
r2 vs mean model 0.9687928467395628
EV 0.9692528247509095
q 1/50
(772,) (756,)
(772,) (756,)
(772,) (756,)
(772,) (756,)
(772,) (756,)
r2 vs mean model 0.9682546738611733
EV 0.9687225842746937
q 2/50
(772,) (741,)
(772,) (741,)
(772,) (741,)
(772,) (741,)
(772,) (741,)
r2 vs mean model 0.9682744869349902
EV 0.9687421053135975
q 3/50
(772,) (725,)
(772,) (725,)
(772,) (725,)
(772,) (725,)
(772,) (725,)
r2 vs mean model 0.9691409722066212
EV 0.9695958190206844
q 4/50
(772,) (710,)
(772,) (710,)
(772,) (710,)
(772,) (710,)
(772,) (710,)
r2 vs mean model 0.9693228946834931
EV 0.9697750600501832
q 5/50
(772,) (694,)
(772,) (694,)
(772,) (694,)
(772,) (694,)
(772,) (694,)
r2 vs mean model 0.9699398947752269
EV 0.9703829658655878
q 6/50
(772,) (679,)
(772,) (679,)
(772,) (679,)
(77

r2 vs mean model 0.5239546000228387
EV 0.525415532270567
q 12/50
(772,) (586,)
(772,) (586,)
(772,) (586,)
(772,) (586,)
(772,) (586,)
r2 vs mean model 0.5201337936051382
EV 0.5216064514977488
q 13/50
(772,) (571,)
(772,) (571,)
(772,) (571,)
(772,) (571,)
(772,) (571,)
r2 vs mean model 0.5190030048169207
EV 0.5204791329789765
q 14/50
(772,) (556,)
(772,) (556,)
(772,) (556,)
(772,) (556,)
(772,) (556,)
r2 vs mean model 0.5170057905405356
EV 0.518488047938883
q 15/50
(772,) (540,)
(772,) (540,)
(772,) (540,)
(772,) (540,)
(772,) (540,)
r2 vs mean model 0.5140196079808965
EV 0.5155110296530698
q 16/50
(772,) (525,)
(772,) (525,)
(772,) (525,)
(772,) (525,)
(772,) (525,)
r2 vs mean model 0.5108203138610865
EV 0.5123215538235948
q 17/50
(772,) (509,)
(772,) (509,)
(772,) (509,)
(772,) (509,)
(772,) (509,)
r2 vs mean model 0.5100413420164316
EV 0.511544972559939
q 18/50
(772,) (494,)
(772,) (494,)
(772,) (494,)
(772,) (494,)
(772,) (494,)
r2 vs mean model 0.5086590251401111
EV 0.5101668978

r2 vs mean model 0.6372416449350817
EV 0.6343277576179772
q 22/50
(772,) (432,)
(772,) (432,)
(772,) (432,)
(772,) (432,)
(772,) (432,)
r2 vs mean model 0.6358078358819146
EV 0.632882431371734
q 23/50
(772,) (417,)
(772,) (417,)
(772,) (417,)
(772,) (417,)
(772,) (417,)
r2 vs mean model 0.6327649491407598
EV 0.6298151023842227
q 24/50
(772,) (401,)
(772,) (401,)
(772,) (401,)
(772,) (401,)
(772,) (401,)
r2 vs mean model 0.617816920932738
EV 0.6147470028692193
q 25/50
(772,) (386,)
(772,) (386,)
(772,) (386,)
(772,) (386,)
(772,) (386,)
r2 vs mean model 0.6483603462206571
EV 0.6455357708165832
q 26/50
(772,) (371,)
(772,) (371,)
(772,) (371,)
(772,) (371,)
(772,) (371,)
r2 vs mean model 0.6550229528289346
EV 0.6522518953786822
q 27/50
(772,) (355,)
(772,) (355,)
(772,) (355,)
(772,) (355,)
(772,) (355,)
r2 vs mean model 0.6557207864587224
EV 0.6529553344164087
q 28/50
(772,) (340,)
(772,) (340,)
(772,) (340,)
(772,) (340,)
(772,) (340,)
r2 vs mean model 0.6525214780777502
EV 0.649730327

r2 vs mean model 0.4192031759018313
EV 0.4235065552287022
q 34/50
(772,) (247,)
(772,) (247,)
(772,) (247,)
(772,) (247,)
(772,) (247,)
r2 vs mean model 0.43386867604917045
EV 0.43806339222993984
q 35/50
(772,) (232,)
(772,) (232,)
(772,) (232,)
(772,) (232,)
(772,) (232,)
r2 vs mean model 0.4427385871995394
EV 0.44686758230420787
q 36/50
(772,) (216,)
(772,) (216,)
(772,) (216,)
(772,) (216,)
(772,) (216,)
r2 vs mean model 0.43812944096492434
EV 0.44229258726291487
q 37/50
(772,) (201,)
(772,) (201,)
(772,) (201,)
(772,) (201,)
(772,) (201,)
r2 vs mean model 0.42821693080407286
EV 0.43245352325316766
q 38/50
(772,) (186,)
(772,) (186,)
(772,) (186,)
(772,) (186,)
(772,) (186,)
r2 vs mean model 0.4277539844703556
EV 0.43199400709261826
q 39/50
(772,) (170,)
(772,) (170,)
(772,) (170,)
(772,) (170,)
(772,) (170,)
r2 vs mean model 0.38886853682399725
EV 0.3933966788446007
q 40/50
(772,) (155,)
(772,) (155,)
(772,) (155,)
(772,) (155,)
(772,) (155,)
r2 vs mean model 0.39117580987114453
EV

(772,) (78,)
(772,) (78,)
(772,) (78,)
(772,) (78,)
(772,) (78,)
r2 vs mean model 0.5163998837717747
EV 0.5212237673768287
q 46/50
(772,) (62,)
(772,) (62,)
(772,) (62,)
(772,) (62,)
(772,) (62,)
r2 vs mean model 0.5145565844607424
EV 0.5193988548713974
q 47/50
(772,) (47,)
(772,) (47,)
(772,) (47,)
(772,) (47,)
(772,) (47,)
r2 vs mean model 0.5205222867488004
EV 0.5253050496190428
q 48/50
(772,) (31,)
(772,) (31,)
(772,) (31,)
(772,) (31,)
(772,) (31,)
r2 vs mean model 0.5076891451275396
EV 0.5125999178542537
q 49/50
(772,) (16,)
(772,) (16,)
(772,) (16,)
(772,) (16,)
(772,) (16,)
r2 vs mean model 0.5164146108924536
EV 0.521238347595318

